In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime
pd.set_option('display.max_columns', 5000)

In [43]:
# !curl https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/counties.csv  -o ../data/raw/counties_09-11.csv
# !curl https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/deaths_timeseries.csv -o ../data/raw/time_series_covid19_deaths_US_06-19.csv
# !curl https://opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D -o ../data/raw/hospitals.csv
# !curl https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-m50.csv -o ../data/raw/DL-us-m50.csv
# !curl https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-m50_index.csv -o ../data/raw/DL-us-m50_index.csv
# !curl https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv -o ../data/raw/time_series_covid19_deaths_US_05-22.csv
# !curl https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=694ae9957380f150 -o ../data/raw/google_mobility_report_06-14-20.csv
# !curl https://raw.githubusercontent.com/HopkinsIDD/hit-covid/master/data/hit-covid-longdata.csv -o ../data/raw/hit_covid_interventions_06-26-20.csv
# !curl https://raw.githubusercontent.com/Keystone-Strategy/covid19-intervention-data/master/complete_npis_raw_policies.csv -o ../data/raw/complete_npis_inherited_policies_keystone_06_26.csv # have to delete first cell/ttitle
# !curl https://raw.githubusercontent.com/COVID19StatePolicy/SocialDistancing/master/data/USstatesCov19distancingpolicy.csv -o ../data/raw/USstatesCov19distancingpolicy_07_01.csv
# !curl https://data.cms.gov/resource/s2uc-8wxp.csv -o ../data/raw/longtermcare_deaths_07_08.csv

In [44]:
hospitals = pd.read_csv('../data/raw/hospitals.csv')
hospitals.columns

Index(['FID', 'ID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4',
       'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION', 'COUNTY', 'COUNTYFIPS',
       'COUNTRY', 'LATITUDE', 'LONGITUDE', 'NAICS_CODE', 'NAICS_DESC',
       'SOURCE', 'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'STATE_ID',
       'ALT_NAME', 'ST_FIPS', 'OWNER', 'TTL_STAFF', 'BEDS', 'TRAUMA',
       'HELIPAD'],
      dtype='object')

## Pull Hospital Data

### Long-term Care Hospitals

In [45]:
longterm_all = pd.read_csv('../data/raw/Long-_Term_Care_Hospital_-_General_Information.csv')
longterm_all['Long-term beds'] = longterm_all['Total Number of Beds']
longterm_all = longterm_all.drop(columns='Total Number of Beds')
fipscodes = pd.read_csv('../data/raw/countyfipstool2019.csv').filter(['sab', 'cname', 'fips'])
longterm_all = fipscodes.merge(longterm_all, left_on=['sab', 'cname'], right_on=['State', 'County Name'])
longterm_all['FIPS'] = longterm_all['fips']
longterm_all = longterm_all.drop(columns=['sab', 'cname', 'fips', 'State', 'County Name'])
longterm = longterm_all.filter(['FIPS', 'Long-term beds', 'State', 'County Name'])
longterm = longterm.dropna()
longterm.head()

,FIPS,Long-term beds
0,1015,38
1,1069,35
2,1073,38
3,1073,45
4,1083,31


In [46]:
fips_groups =longterm.groupby("FIPS")

LT_hosp_data = pd.DataFrame(columns=['FIPS', 'Longterm HospCt', 'Longterm Beds'])
for fips_code, grp in fips_groups:
    LT_hosp_data = LT_hosp_data.append({'FIPS': fips_code, 'Longterm HospCt': len(grp), 'Longterm Beds': sum(grp['Long-term beds'])}, ignore_index=True)   

LT_hosp_data.head()

,FIPS,Longterm HospCt,Longterm Beds
0,1015,1,38
1,1069,1,35
2,1073,2,83
3,1083,1,31
4,1097,1,22


### Nursing Homes

In [47]:
nursing_homes = pd.read_csv('../data/raw/Nursing_Homes.csv').filter(['COUNTYFIPS','BEDS', 'STATUS'])
nursing_homes = nursing_homes[nursing_homes['STATUS'] == 'OPEN']
nursing_homes = nursing_homes.drop(columns='STATUS')
nursing_homes.head()

,COUNTYFIPS,BEDS
0,54019,58
1,51153,107
2,37051,170
3,25017,-999
4,55035,6


In [48]:
nursing = pd.DataFrame(columns=['FIPS', 'NursingCt', 'NursingBeds'])
nursing_fips = nursing_homes.groupby('COUNTYFIPS')
for fips, grp in nursing_fips:
    nursing = nursing.append({'FIPS': fips, 'NursingCt': len(grp), 'NursingBeds': sum(grp['BEDS'])}, ignore_index=True)
nursing.head()

,FIPS,NursingCt,NursingBeds
0,1001,6,315
1,1003,25,1816
2,1005,3,212
3,1007,1,131
4,1009,9,367


### General Hospitals

In [49]:
# Created by Michael
hospitals = pd.read_csv('../data/raw/hospitals.csv')[['TYPE', 'STATUS', 'COUNTYFIPS', 'BEDS', 'STATE', 'COUNTY']]
print(set(hospitals["TYPE"]))
hospitals = hospitals[hospitals["STATUS"] == 'OPEN']
hospitals = hospitals[hospitals["TYPE"] == 'GENERAL ACUTE CARE']
hospitals = hospitals[hospitals["BEDS"].astype(str).astype(int) > 0]
hospitals["FIPS"] = hospitals["COUNTYFIPS"]
hospitals = hospitals[hospitals["FIPS"] != 'NOT AVAILABLE']
hospitals = hospitals.drop(["COUNTYFIPS", "STATUS"], axis=1)
hospitals["FIPS"] = hospitals["FIPS"].astype(str).astype(int)

fips_groups = hospitals.groupby("FIPS")

hosp_data = pd.DataFrame(columns=['FIPS', 'HospCt', 'Beds'])
for fips_code, grp in fips_groups:
    hosp_data = hosp_data.append({'FIPS': fips_code, 'HospCt': len(grp), 'Beds': sum(grp["BEDS"])}, ignore_index=True)   

hosp_data.head()

{'CHILDREN', 'MILITARY', 'REHABILITATION', 'SPECIAL', 'LONG TERM CARE', 'CHRONIC DISEASE', 'GENERAL ACUTE CARE', 'PSYCHIATRIC', 'CRITICAL ACCESS', 'WOMEN'}


,FIPS,HospCt,Beds
0,1001,1,85
1,1003,3,332
2,1005,1,74
3,1007,1,35
4,1011,1,61


## County Data

In [50]:
counties = pd.read_csv('../data/raw/counties_09-11.csv')
# print(counties.columns.values)
# counties = counties.dropna(how='any')
# counties['FIPS'] = counties['FIPS'] .astype('int')
counties = counties.set_index('FIPS')
# print(counties.where(counties['FIPS'] == 36061))
# print(counties.loc[6037,], '\n')
# counties = counties.filter(['FIPS', 'Rural-urban_Continuum Code_2013', 'Density per square mile of land area - Population'])
counties = counties.filter(['FIPS',  'Density per square mile of land area - Population', 'Rural-urban_Continuum Code_2013', \
                            'Percent of adults with less than a high school diploma 2014-18',\
                            "Percent of adults with a bachelor's degree or higher 2014-18", 'Unemployment_rate_2018', \
                            'Med_HH_Income_Percent_of_State_Total_2018', 'Jan Temp AVG / F', 'Feb Temp AVG / F', 'Mar Temp AVG / F', 'Apr Temp AVG / F', 'May Temp AVG / F', \
                            'Jun Temp AVG / F', 'Jul Temp AVG / F', 'Aug Temp AVG / F', 'Sep Temp AVG / F', 'Oct Temp AVG / F', 'Nov Temp AVG / F', 'Dec Temp AVG / F', 'Total_age65plus',  'Total households!!Average household size',\
                            'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool',\
                            'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten',\
                            'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8)',\
                            'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)',\
                            'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school', 'POP_ESTIMATE_2018', \
                            'H_MALE', 'H_FEMALE', 'BA_MALE', 'BA_FEMALE', 'Area in square miles - Land area'])
print(counties.shape)
# counties = counties.dropna()
print(counties.shape)
counties = counties.merge(hosp_data, how='left', left_on='FIPS', right_on='FIPS')
counties = counties.merge(LT_hosp_data, how='left',  left_on='FIPS', right_on='FIPS')
counties = counties.merge(nursing, how='left', left_on='FIPS', right_on='FIPS')
print("test:", counties.shape)
# counties = counties.fillna(value=0) ### 
counties[['HospCt', 'Beds', 'NursingCt', 'NursingBeds','Longterm HospCt', 'Longterm Beds']] = counties[['HospCt', 'Beds', 'NursingCt', 'NursingBeds', 'Longterm HospCt', 'Longterm Beds']].fillna(value=0)
print(counties.shape)

counties['HospCt'] = counties['HospCt']
print('range Hosp Ct: ', counties['HospCt'].min(axis=0), np.mean(counties['HospCt'], axis=0), np.median(counties['HospCt'], axis=0), np.percentile(counties['HospCt'], 75, axis=0), counties['HospCt'].max(axis=0))
print(counties.shape)
#print('num above 800', counties['HospCt'][counties['HospCt'] > 800].shape)
# cutoff hopsitals at 75th percentile
hospital_cutoff = np.percentile(counties['HospCt'], 75, axis=0)
counties.loc[counties['HospCt'] > hospital_cutoff, 'HospCt'] = hospital_cutoff
print('range hosp beds: ', counties['Beds'].min(axis=0), counties['Beds'].max(axis=0))
print('range hosp beds: ', counties['HospCt'].min(axis=0), counties['HospCt'].max(axis=0))


print('range Nursing Ct: ', counties['NursingCt'].min(axis=0), np.percentile(counties['NursingCt'], 25, axis=0), np.mean(counties['NursingCt']), np.median(counties['NursingCt']), np.percentile(counties['NursingCt'], 75, axis=0), counties['NursingCt'].max(axis=0))
print('range NursingBeds: ', counties['NursingBeds'].min(axis=0), np.percentile(counties['NursingBeds'], 25, axis=0), np.mean(counties['NursingBeds']), np.median(counties['NursingBeds']), np.percentile(counties['NursingBeds'], 75, axis=0), counties['NursingBeds'].max(axis=0))
counties['NursingCt'] = counties['NursingCt'] + counties['Longterm HospCt']
print('range Nursing Ct: ', counties['NursingCt'].min(axis=0), np.percentile(counties['NursingCt'], 25, axis=0), np.mean(counties['NursingCt']), np.median(counties['NursingCt']), np.percentile(counties['NursingCt'], 75, axis=0), counties['NursingCt'].max(axis=0))
print('range NursingBeds: ', counties['NursingBeds'].min(axis=0), np.percentile(counties['NursingBeds'], 25, axis=0), np.mean(counties['NursingBeds']), np.median(counties['NursingBeds']), np.percentile(counties['NursingBeds'], 75, axis=0), counties['NursingBeds'].max(axis=0))

nursing_cutoff = np.percentile(counties['NursingCt'], 75, axis=0)

# cut off nursing home count at 75th percentile 
counties.loc[counties['NursingCt'] > nursing_cutoff, 'NursingCt'] = nursing_cutoff

print('range Nursing Ct: ', counties['NursingCt'].min(axis=0), np.percentile(counties['NursingCt'], 25, axis=0), np.mean(counties['NursingCt']), np.median(counties['NursingCt']), np.percentile(counties['NursingCt'], 75, axis=0), counties['NursingCt'].max(axis=0))

counties = counties.drop(columns=['Longterm HospCt'])
counties = counties.drop(columns=['Longterm Beds'])
counties = counties.drop(columns=['NursingBeds'])

counties['Hispanic Population'] = counties['H_MALE'] + counties['H_FEMALE']
counties = counties.drop(columns=['H_FEMALE', 'H_MALE'])
counties['Black Population'] = counties['BA_MALE'] + counties['BA_FEMALE']
counties = counties.drop(columns=['BA_FEMALE', 'BA_MALE'])
print(counties.shape)
counties.head()
# 187 

(3273, 31)
(3273, 31)
test: (3273, 38)
(3273, 38)
range Hosp Ct:  0 1.2343415826458906 1.0 1.0 103
(3273, 38)
range hosp beds:  0 24676
range hosp beds:  0.0 1.0
range Nursing Ct:  0 2.0 12.047662694775436 4.0 9.0 1662
range NursingBeds:  -197527 0.0 -699.7849068133211 159.0 437.0 74443
range Nursing Ct:  0 2.0 12.15948670944088 4.0 9.0 1666
range NursingBeds:  -197527 0.0 -699.7849068133211 159.0 437.0 74443
range Nursing Ct:  0.0 2.0 4.466238924534067 4.0 9.0 9.0
(3273, 33)


,FIPS,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Jan Temp AVG / F,Feb Temp AVG / F,Mar Temp AVG / F,Apr Temp AVG / F,May Temp AVG / F,Jun Temp AVG / F,Jul Temp AVG / F,Aug Temp AVG / F,Sep Temp AVG / F,Oct Temp AVG / F,Nov Temp AVG / F,Dec Temp AVG / F,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,Area in square miles - Land area,HospCt,Beds,NursingCt,Hispanic Population,Black Population
0,0,87.4,NaN,12.3,31.5,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52431193.0,0.00,4300436.0,3397087.0,27916769.0,14550764.0,19757430.0,327167434,3531905.43,0.0,0,0.0,59722584.0,43799095.0
1,1000,94.4,NaN,14.2,24.9,3.9,100.0,46.06,55.57,54.56,63.26,73.92,77.8,80.4,80.34,79.93,67.48,50.28,50.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4887871,50645.33,0.0,0,0.0,NaN,NaN
2,1001,91.8,2.0,11.3,27.7,3.6,119.0,47.00,58.60,56.10,65.00,75.50,79.3,81.5,81.80,81.60,69.30,51.70,52.00,8653.0,NaN,NaN,NaN,NaN,NaN,NaN,55601,594.44,1.0,85,6.0,1649.0,10915.0
3,1003,114.6,3.0,9.7,31.3,3.6,115.5,52.00,62.00,59.70,66.00,76.50,81.3,82.5,82.50,82.10,72.30,56.20,56.10,44571.0,2.57,3065.0,1449.0,23254.0,11209.0,6503.0,218022,1589.78,1.0,332,9.0,10131.0,19492.0
4,1005,31.0,6.0,27.0,12.2,5.2,68.9,49.00,58.50,57.40,64.90,75.20,79.6,81.3,81.10,80.60,70.90,52.80,51.60,4832.0,NaN,NaN,NaN,NaN,NaN,NaN,24881,884.88,1.0,74,3.0,1064.0,12042.0


## Deaths Data

In [51]:
deaths_t_series = pd.read_csv('../data/raw/time_series_covid19_deaths_US_06-19.csv')
# print(filtered.loc[36061,])
# print(deaths.columns.values)
# deaths_t_series = deaths_t_series.drop(['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Population', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'], axis=1)
deaths_t_series = deaths_t_series.drop(['Combined_Key'], axis=1)

# deaths = deaths.dropna()
# deaths = deaths.set_index('FIPS')

deaths_t_series.head()

,FIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
0,1001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,2,2,2,1,1,2,2,2,2,2,3,4,4,4,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,3,3,3,3,3,4,4,5,5,5,5,5,5,5,5,5,6,6,6,6,6,6
1,1003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,4,4,4,4,5,5,5,5,5,5,6,7,7,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
2,1005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1009,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [52]:
# total_deaths = pd.DataFrame(deaths.sum(axis=1), columns=['deaths'])
# total_deaths.columns = ['FIPS', 'deaths']
death_data = pd.DataFrame(columns=['FIPS', 'Deaths', 'Outbreak_date'])
dates = deaths_t_series.columns[1:]
death_thresh = 3
n_days = 21

# Get all existing data n_days from the death_thresh death.
for i in range(len(deaths_t_series)):
    county = deaths_t_series.iloc[i]
    outbreak_date = None

    # Iterate through dates to find first date with deaths >= death thresh
    for j in range(len(dates)):
        if county[dates[j]] >= death_thresh: 
            outbreak_date = dates[j]

            # See if this happened n_days or more before current day
            try:
                n_days_later = dates[j+n_days]

                # Make sure the data is bug-free
                bad_data=False
                for k in range(n_days):
                    if county[n_days_later] < county[dates[j+k]]:
                        bad_data=True
                if not bad_data:
                    outbreak_date = datetime.datetime.strptime(outbreak_date, '%m/%d/%y')
                    death_data = death_data.append({'FIPS': county['FIPS'], 'Deaths':county[n_days_later], 'Outbreak_date':outbreak_date}, ignore_index=True)
                break

            except:
                continue
#                 break
                
death_data.head()

,FIPS,Deaths,Outbreak_date
0,1001,4,2020-04-27
1,1003,6,2020-04-20
2,1011,8,2020-05-25
3,1013,16,2020-05-08
4,1015,3,2020-04-20


In [53]:
combined_data = counties.merge(death_data, left_on='FIPS', right_on='FIPS')
# combined_data = combined_data.dropna()
combined_data.set_index('FIPS', inplace=True)
print(combined_data.shape)
combined_data.head()

(1007, 34)


,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Jan Temp AVG / F,Feb Temp AVG / F,Mar Temp AVG / F,Apr Temp AVG / F,May Temp AVG / F,Jun Temp AVG / F,Jul Temp AVG / F,Aug Temp AVG / F,Sep Temp AVG / F,Oct Temp AVG / F,Nov Temp AVG / F,Dec Temp AVG / F,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,Area in square miles - Land area,HospCt,Beds,NursingCt,Hispanic Population,Black Population,Deaths,Outbreak_date
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,91.8,2.0,11.3,27.7,3.6,119.0,47.0,58.6,56.1,65.0,75.5,79.3,81.5,81.8,81.6,69.3,51.7,52.0,8653.0,NaN,NaN,NaN,NaN,NaN,NaN,55601,594.44,1.0,85,6.0,1649.0,10915.0,4,2020-04-27
1003,114.6,3.0,9.7,31.3,3.6,115.5,52.0,62.0,59.7,66.0,76.5,81.3,82.5,82.5,82.1,72.3,56.2,56.1,44571.0,2.57,3065.0,1449.0,23254.0,11209.0,6503.0,218022,1589.78,1.0,332,9.0,10131.0,19492.0,6,2020-04-20
1011,17.5,6.0,24.8,13.3,4.7,58.7,48.0,58.2,56.6,64.3,74.9,79.1,80.9,80.7,80.5,70.0,52.3,51.6,1661.0,NaN,NaN,NaN,NaN,NaN,NaN,10138,622.80,1.0,61,1.0,807.0,7168.0,8,2020-05-25
1013,27.0,6.0,15.4,16.1,4.8,74.9,48.4,59.3,57.0,64.4,75.1,79.1,81.0,81.0,81.2,70.2,52.6,53.2,3995.0,NaN,NaN,NaN,NaN,NaN,NaN,19680,776.83,1.0,94,5.0,297.0,8839.0,16,2020-05-08
1015,195.7,3.0,15.9,18.0,4.7,91.0,44.2,52.6,53.0,63.0,72.9,76.2,79.6,79.9,78.8,66.2,49.3,49.9,20247.0,2.51,1563.0,1484.0,11224.0,4972.0,6890.0,114277,605.87,1.0,552,9.0,4469.0,24153.0,3,2020-04-20


## Longterm Care Deaths

In [13]:
# print(longterm_all.loc[0, ])
# print(longterm_all.shape)
# longterm_all.head()

In [14]:
# longterm_deaths = pd.read_csv('../data/raw/longtermcare_deaths_07_08.csv')
# longterm_deaths['CMS Certification Number (CCN)'] = np.nan
# print(longterm_deaths.loc[0, 'federal_provider_number'])
# print(int(longterm_deaths.loc[0, 'federal_provider_number']))
# for i in longterm_deaths.index:
#     try: 
#         longterm_deaths.loc[i, 'CMS Certification Number (CCN)'] = int(longterm_deaths.loc[i, 'federal_provider_number'])
#     except:
#         longterm_deaths.drop(index=i, inplace=True)
#         continue
# # longterm_deaths = longterm_deaths.dropna()        
# longterm_deaths['CMS Certification Number (CCN)'] = longterm_deaths['CMS Certification Number (CCN)'].astype('int64')
# print(longterm_deaths.shape)
# longterm_deaths.head(15)

In [15]:
# for i in longterm_deaths['CMS Certification Number (CCN)'].values: 
#     if i in longterm_all['CMS Certification Number (CCN)'].values:
#         print(i)
# print(longterm_all['CMS Certification Number (CCN)'].values)

In [16]:
# print(longterm_deaths['CMS Certification Number (CCN)'].values)

In [17]:
# print(longterm_deaths['provider_name'].values)

In [18]:

# longterm_all = longterm_all.merge(longterm_deaths, left_on=['CMS Certification Number (CCN)'], right_on=['CMS Certification Number (CCN)'], how='left')
# print(longterm_all.shape)
# longterm_all = longterm_all.dropna()
# print(longterm_all.shape)
# longterm_all.head()

## Drop Outlier Counties

In [54]:
combined_data.head()
print(len(combined_data))

1007


## Mobility Data

In [20]:
# dl_index = pd.read_csv('../data/raw/DL-us-m50_index_05-22.csv')
# dl_abs_movement = pd.read_csv('../data/raw/DL-us-m50_05-22.csv')
# dl_index = dl_index.dropna()
# dl_abs_movement = dl_abs_movement.dropna()
# dl = dl_index.merge(dl_abs_movement, on='fips')
# print(dl.head())

# baseline1 = pd.DataFrame(dl['2020-03-01_y'] / (.01 * dl['2020-03-01_x']))
# baseline2 = pd.DataFrame(dl['2020-03-02_y'] / (.01 * dl['2020-03-02_x']))
# baseline3 = pd.DataFrame(dl['2020-03-03_y'] / (.01 * dl['2020-03-03_x']))
# baselines = pd.concat((baseline1, baseline2, baseline3), axis=1)

# pd.set_option('display.max_rows', None)
# dl['baseline m50 mobility'] = baselines.mean(axis=1)
# baseline_data = dl[['baseline m50 mobility', 'fips']]
# baseline_data['FIPS'] = baseline_data['fips']
# baseline_data = baseline_data.drop(['fips'], axis=1)
# baseline_data.set_index('FIPS', inplace=True)
# baseline_data.head()

In [21]:
# combined_data = baseline_data.merge(combined_data, left_on='FIPS', right_on='FIPS')
# combined_data.head()

## Save Data

In [55]:
print(combined_data.shape)
combined_data.to_csv('../data/intermediates/pre_safegraph.csv')

(1007, 34)


In [56]:
sg_mobility = pd.read_csv('../data/processed/safegraph_mobility_avgs_ml.csv')

combined_data = sg_mobility.merge(combined_data, left_on='FIPS', right_on='FIPS')
print(combined_data.shape)
combined_data.head()


(1007, 39)


,FIPS,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Jan Temp AVG / F,Feb Temp AVG / F,Mar Temp AVG / F,Apr Temp AVG / F,May Temp AVG / F,Jun Temp AVG / F,Jul Temp AVG / F,Aug Temp AVG / F,Sep Temp AVG / F,Oct Temp AVG / F,Nov Temp AVG / F,Dec Temp AVG / F,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,Area in square miles - Land area,HospCt,Beds,NursingCt,Hispanic Population,Black Population,Deaths,Outbreak_date
0,1001,7461.571429,7314.285714,13927.142857,13194.857143,91.8,2.0,11.3,27.7,3.6,119.0,47.0,58.6,56.1,65.0,75.5,79.3,81.5,81.8,81.6,69.3,51.7,52.0,8653.0,NaN,NaN,NaN,NaN,NaN,NaN,55601,594.44,1.0,85,6.0,1649.0,10915.0,4,2020-04-27
1,1003,35004.000000,32185.714286,47460.285714,42444.714286,114.6,3.0,9.7,31.3,3.6,115.5,52.0,62.0,59.7,66.0,76.5,81.3,82.5,82.5,82.1,72.3,56.2,56.1,44571.0,2.57,3065.0,1449.0,23254.0,11209.0,6503.0,218022,1589.78,1.0,332,9.0,10131.0,19492.0,6,2020-04-20
2,1011,682.142857,690.714286,1401.285714,1447.571429,17.5,6.0,24.8,13.3,4.7,58.7,48.0,58.2,56.6,64.3,74.9,79.1,80.9,80.7,80.5,70.0,52.3,51.6,1661.0,NaN,NaN,NaN,NaN,NaN,NaN,10138,622.80,1.0,61,1.0,807.0,7168.0,8,2020-05-25
3,1013,2333.142857,2367.285714,4201.714286,4058.285714,27.0,6.0,15.4,16.1,4.8,74.9,48.4,59.3,57.0,64.4,75.1,79.1,81.0,81.0,81.2,70.2,52.6,53.2,3995.0,NaN,NaN,NaN,NaN,NaN,NaN,19680,776.83,1.0,94,5.0,297.0,8839.0,16,2020-05-08
4,1015,16956.857143,16182.142857,25290.571429,23753.571429,195.7,3.0,15.9,18.0,4.7,91.0,44.2,52.6,53.0,63.0,72.9,76.2,79.6,79.9,78.8,66.2,49.3,49.9,20247.0,2.51,1563.0,1484.0,11224.0,4972.0,6890.0,114277,605.87,1.0,552,9.0,4469.0,24153.0,3,2020-04-20


In [57]:
combined_data.to_csv('../data/processed/ml_dataset.csv')